# Proyecto final de Aprendizaje Automático
## Autor:
Daniel Useche.
ignaciousecheparedes@gmail.com
---

## Descripcion:

Análisis de la aplicación de multiclasificadores tipo Bagging y Boosting a la data de pacientes diabéticos extraida de 130 hospitales de los Estados Unidos en el periodo comprendido entre 1999-2008.

El data set original puede encontrarse en el siguiente [enlace](https://archive.ics.uci.edu/ml/datasets/Diabetes+130-US+hospitals+for+years+1999-2008#) o en la dirección: https://archive.ics.uci.edu/ml/datasets/Diabetes+130-US+hospitals+for+years+1999-2008#

Para una explicación detallada de los atributos presentes en el dataset dirigirse a https://www.hindawi.com/journals/bmri/2014/781670/

---

## Cargando el DataSet 

In [1]:
import numpy as np
import pandas as pd 
diabeticDataSet = pd.read_csv("diabetic_data.csv", header = 0)
diabeticDataSet.tail()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
101761,443847548,100162476,AfricanAmerican,Male,[70-80),?,1,3,7,3,...,No,Down,No,No,No,No,No,Ch,Yes,>30
101762,443847782,74694222,AfricanAmerican,Female,[80-90),?,1,4,5,5,...,No,Steady,No,No,No,No,No,No,Yes,NO
101763,443854148,41088789,Caucasian,Male,[70-80),?,1,1,7,1,...,No,Down,No,No,No,No,No,Ch,Yes,NO
101764,443857166,31693671,Caucasian,Female,[80-90),?,2,3,7,10,...,No,Up,No,No,No,No,No,Ch,Yes,NO
101765,443867222,175429310,Caucasian,Male,[70-80),?,1,1,7,6,...,No,No,No,No,No,No,No,No,No,NO


---
## Preprocesado:

### Eliminación de caracteristicas ruidosas, o con valores faltantes:
Como todo conjunto de datos, el data set utilizado tiene atributos que poseen elementos faltantes en su mayoria, redundantes o ruidosos, como se muestra en la tabla 1 (pág 3) del [documento](https://www.hindawi.com/journals/bmri/2014/781670/) que describe los atributos, los siguientes atributos deben ser depurados del dataset por su cantidad de valores faltantes:
- weight: indica el peso del paciente, pero como su medida no es obligatoria, posee un aproximado de 97% de valores faltantes.
- payer_code: número que indica el tipo de pago realizado por el paciente, 40% de datos faltantes e irrelevante para el análisis.
- medical_specialty: indica la especialidad del médico que brindó la atención, posee 49% de datos faltantes.

In [2]:
diabeticDataSet = diabeticDataSet.drop(columns = ['weight', 'payer_code', 'medical_specialty'])

### Estableciendo un registro único por paciente:
Tambien hay que tomar en cuenta que puede existir mas de un registro para el mismo paciente, por lo que se tomará el atributo **patient_nbr** para filtrar los encuentros y así no existan pacientes duplicados:

In [3]:
diabeticDataSet = diabeticDataSet.drop_duplicates(subset='patient_nbr')

### Vista preliminar del dataset:

In [4]:
diabeticDataSet

,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),6,25,1,1,41,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),1,1,7,3,59,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),1,1,7,2,11,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),1,1,7,2,44,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),1,1,7,1,51,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101754,443842016,183087545,Caucasian,Female,[70-80),1,1,7,9,50,...,No,Steady,No,No,No,No,No,Ch,Yes,>30
101755,443842022,188574944,Other,Female,[40-50),1,1,7,14,73,...,No,Up,No,No,No,No,No,Ch,Yes,>30
101756,443842070,140199494,Other,Female,[60-70),1,1,7,2,46,...,No,Steady,No,No,No,No,No,No,Yes,>30
101758,443842340,120975314,Caucasian,Female,[80-90),1,1,7,5,76,...,No,Up,No,No,No,No,No,Ch,Yes,NO


### Transformación de valores faltantes con el formato '?' a un valor compatible Nan
La Mayoría de los algoritmos de aprendizaje requieren que sus entradas no posean valores faltantes, y no funcionarán si este requerimiento es violado [ref](https://scikit-learn.org/stable/modules/impute.html#impute).

In [14]:
diabeticDataSet = diabeticDataSet.replace('?', np.nan) 

,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),6,25,1,1,41,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),1,1,7,3,59,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),1,1,7,2,11,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),1,1,7,2,44,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),1,1,7,1,51,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101754,443842016,183087545,Caucasian,Female,[70-80),1,1,7,9,50,...,No,Steady,No,No,No,No,No,Ch,Yes,>30
101755,443842022,188574944,Other,Female,[40-50),1,1,7,14,73,...,No,Up,No,No,No,No,No,Ch,Yes,>30
101756,443842070,140199494,Other,Female,[60-70),1,1,7,2,46,...,No,Steady,No,No,No,No,No,No,Yes,>30
101758,443842340,120975314,Caucasian,Female,[80-90),1,1,7,5,76,...,No,Up,No,No,No,No,No,Ch,Yes,NO


In [6]:
from sklearn.preprocessing import OneHotEncoder

In [7]:
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(diabeticDataSet)
enc.categories_

[array([    12522,     15738,     16680, ..., 443842070, 443842340,
        443867222]),
 array([      135,       378,       729, ..., 189445127, 189481478,
        189502619]),
 array(['?', 'AfricanAmerican', 'Asian', 'Caucasian', 'Hispanic', 'Other'],
       dtype=object),
 array(['Female', 'Male', 'Unknown/Invalid'], dtype=object),
 array(['[0-10)', '[10-20)', '[20-30)', '[30-40)', '[40-50)', '[50-60)',
        '[60-70)', '[70-80)', '[80-90)', '[90-100)'], dtype=object),
 array([1, 2, 3, 4, 5, 6, 7, 8]),
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 22, 23, 24, 25, 27, 28]),
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 13, 14, 17, 20, 22, 25]),
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]),
 array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
         27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
  